<a href="https://colab.research.google.com/github/Ron-Rocks/ImageClassification/blob/master/ImageClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import keras
from keras.datasets import cifar10
from keras.layers import Dense,Flatten
from keras.models import Model
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16,preprocess_input
(trainX,trainY),(testX,testY) = cifar10.load_data()

labels = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
trainY = to_categorical(trainY)
testY = to_categorical(testY)
print(trainY.shape)
trainX = trainX/255
trainY = trainY/255

print(trainX.shape)

(50000, 10)
(50000, 32, 32, 3)


In [53]:
model = VGG16(input_shape = (32,32,3,),include_top = False)

for layer in model.layers :
  layer.trainable = False
vggOutput =  model.layers[-2]
x = Flatten()(vggOutput.output)
x = Dense(256,activation = "relu",input_shape=(512,))(x)
x = Dense(10,activation = "sigmoid")(x)

model = Model(model.inputs,x)

model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0  

In [0]:

model.compile(loss = "categorical_crossentropy",optimizer="adam",metrics = ["accuracy"])
model.fit(trainX,trainY,epochs = 10,validation_data=(testX,testY),batch_size=64)
model.save_weights("1weigths.h5")

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
17024/50000 [=========>....................] - ETA: 5:47 - loss: 0.0058 - acc: 0.4778